In [ ]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [ ]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


video_path =  configParser.get('test_imagen','video_path')

ttwbdf =  int(configParser.get('test_imagen', 'time_to_wait_before_deleting_files'))


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [ ]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [ ]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [ ]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [ ]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('test_imagen', 'audio_length_used') 
model_filename =  configParser.get('test_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('test_imagen', 'openl3_mode')
folder =  configParser.get('test_imagen', 'folder')
number_of_images =  configParser.get('test_imagen', 'number_of_images')

In [ ]:
age =  int(configParser.get('test_imagen', 'age'))
ethnicity =  str(configParser.get('test_imagen', 'ethnicity'))
gender =  str(configParser.get('test_imagen', 'gender'))
language =  str(configParser.get('test_imagen', 'language'))
image_guide_path =  str(configParser.get('test_imagen', 'image_guide_path'))
blur_or_pixelate =  int(configParser.get('test_imagen', 'blur_or_pixelate'))

boxBlurMin =  int(configParser.get('test_imagen', 'boxBlurMin'))
boxBlurMax =  int(configParser.get('test_imagen', 'boxBlurMax'))

gaussianBlurMin =  int(configParser.get('test_imagen', 'gaussianBlurMin'))
gaussianBlurMax =  int(configParser.get('test_imagen', 'gaussianBlurMax'))

pix_to_min =  int(configParser.get('test_imagen', 'pix_to_min'))
pix_to_max =  int(configParser.get('test_imagen', 'pix_to_max'))

In [ ]:
dataGotten = [['1','1',video_path,age,ethnicity,gender,[],[],[],language,[]]]
df = pd.DataFrame(dataGotten,columns = ['ID','FACE_ID','VIDEO_PATH','caption_a','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','AUDIO_FEATURES','caption_l','image_path'])
df["image_guide"] = np.nan

In [ ]:
#import tensorflow as tf
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
#  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#gpus
import pickle
import multiprocessing
multiprocessing.set_start_method('spawn')

In [ ]:
from testing_imagen_speechbrain import extract_speechbrain_embeddings
from testing_imagen_pyannote_titanet import extract_pyannote_titanet_embeddings
from testing_imagen_audio_features import extract_audio_features
from testing_imagen_vision_transformer import extract_vision_transformer

audio_embs =  configParser.get('COMMON', 'audio_embs') 
from pydub import AudioSegment
import math
# Get original duration of video
audio = AudioSegment.from_file(video_path)
audio_length_og = math.floor(audio.duration_seconds)


video_filename = os.path.basename(video_path)

absPathAudio_w = os.path.abspath(audio_folder) + "/" + video_filename
absPathAudio = os.path.splitext(absPathAudio_w)[0]+'_audio.wav'
absPathAudio_w = os.path.splitext(absPathAudio_w)[0]


import subprocess
# Extract audio monochannel and with 16khz and put it in absPathAudio

command = "ffmpeg -nostats -loglevel 0 -y -i '" + video_path + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio  + "'"
subprocess.call(command, shell=True)

from threading import Thread
import time

# Function to delete audio temp files
def delFiles(filesToDelete):
    time.sleep(ttwbdf)  # wait a bit
    for file in filesToDelete:  
        try:
            os.remove(file)
        except OSError:
            pass


# Will either truncate or loop the original video to reach audio_length (6,12 or 24)
audio_length_list = [24]
for audio_length in audio_length_list:
    path_var_len_audio =  absPathAudio_w + "audio" + str(audio_length) + "s.wav"    # path to the variable length audio
    path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio

    if(audio_length_og > audio_length):
        # Truncate    

        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)


    else:
        # Loop then truncaate
        #print("lesa")
        twoDigitLenStr = f"{audio_length:02}"
        #print(twoDigitLenStr)
        command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
        subprocess.call(command, shell=True)
        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)




    q = multiprocessing.Queue()

    proc = multiprocessing.Process(target=extract_speechbrain_embeddings, args=(q,path_var_len_audio,))
    proc.start()
    proc.join()



    df['SPEAKER_EMB'] = q.get()
    proc = multiprocessing.Process(target=extract_pyannote_titanet_embeddings, args=(q,path_var_len_audio,audio_embs,audio_length,openl3_mode,))
    proc.start()
    proc.join()

    df['AUDIO_EMB'] = q.get()

    proc = multiprocessing.Process(target=extract_audio_features, args=(q,path_var_len_audio,output_folder,))
    proc.start()
    proc.join()



    with open(output_folder + '/' + 'audio_features.pickle', 'rb') as handle:
        df['AUDIO_FEATURES'] = pickle.load(handle)


    proc = multiprocessing.Process(target=extract_vision_transformer, args=(q,output_folder,
                                                                            image_guide_path,output_folder,
        image_size,boxBlurMin,boxBlurMax,gaussianBlurMin,gaussianBlurMax,pix_to_min,pix_to_max,))
    proc.start()
    proc.join()



    with open(output_folder + '/' + 'image_features.pickle', 'rb') as handle:
        df["image_guide"] = pickle.load(handle)


    

    ftd = [absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp]
    tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
    tDelete.start()



In [ ]:
df3 = df[["image_path","caption_a","caption_e","caption_g","caption_l"]]
data_frame = df3
data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
data_frame['AUDIO_EMB'] = df['AUDIO_EMB']
data_frame['AUDIO_FEATURES'] = df['AUDIO_FEATURES']
data_frame['image_guide'] = df['image_guide']

In [ ]:
data_frame

In [ ]:
import pickle


with open(output_folder + '/' + 'filename.pickle', 'wb') as handle:
    pickle.dump(data_frame, handle)

In [ ]:
import os
os._exit(0)